# Obtener relaciones

Basicamente extraje del bcn las relaciones, en base a que ley modifica a cual
no recuerdo al consulta exacta que use, pero era del estilo:

```
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX bcnnorms: <http://datos.bcn.cl/ontologies/bcn-norms#>

SELECT ?norma ?numero ?titulo ?leycode ?isLatestVersion (GROUP_CONCAT(STR(?modifica); separator=", ") AS ?modificaciones)
WHERE {
  ?norma a bcnnorms:Norm ;
         bcnnorms:type <http://datos.bcn.cl/recurso/cl/norma/tipo#ley> ;
         bcnnorms:hasNumber ?numero ;
         dc:title ?titulo ;
         bcnnorms:leychileCode ?leycode ;
         bcnnorms:isLatestVersion ?isLatestVersion .
  OPTIONAL { ?norma bcnnorms:modifiesTo ?modifica . }
}
GROUP BY ?norma ?numero ?titulo ?leycode ?isLatestVersion

LIMIT 10
OFFSET 10
```

Donde fue ajustando el limit y offset segun correspondia donde obtuve 3 .csv con realciones de modificación.


In [ ]:
import pandas as pd
dfs = []
files = ["relations/1.csv","relations/2.csv","relations/3.csv"]


for file in files:
    df = pd.read_csv(file)
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)
df.to_csv("data/relations.csv")
df.head()

,norma,numero,titulo,leycode,isLatestVersion,modificaciones
0,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,19317,MODIFICA DISPOSICIONES QUE INDICA DE LOS CODIG...,30684,1,http://datos.bcn.cl/recurso/cl/ley/ministerio-...
1,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,15666,"CONCEDE PENSION, POR GRACIA, A DOÑA ENRIQUETA ...",241809,1,NaN
2,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,4808,REFORMA LA LEY SOBRE EL REGISTRO CIVIL,24928,0,NaN
3,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,3534,"LEI NUM. 3,534, QUE CONCEDE FONDOS PARA ADQUIR...",24047,1,NaN
4,http://datos.bcn.cl/recurso/cl/ley/ministerio-...,14936,"RECONOCE, POR GRACIA, EN LA HOJA DE SERVICIOS ...",243083,1,NaN


# Buscar con cuales hacen match

In [ ]:
relations = []

for lawcode in df.leycode:
    rel_to = {lawcode: set()}
    try:
        # Obtener el texto de modificaciones
        mods_str = df.query("leycode == @lawcode")["modificaciones"].iloc[0]

        # saltar nan
        if not isinstance(mods_str, str) or not mods_str.strip():
            continue

        # Limpiar y dividir por comas
        mods = [m.strip() for m in mods_str.split(",") if m.strip()]

        for m in mods:
            # Buscar las leyes asociadas
            laws_associated = df.query("norma == @m")["leycode"].tolist()

            if not laws_associated:
                # no hay relaciones
                # no se encontro la ley
                continue

            for assoc in laws_associated:
                rel_to[lawcode].add(int(assoc))  # Convertir a int

    except Exception as e:
        print(f"e rror con ley {lawcode}: {e}")

    relations.append(rel_to)

In [ ]:
relations

# En el output:
# [{30684: {22740, 22960, 25563}} esto quiere decir 30684 modifica 22740, 22960 y 25563

[{30684: {22740, 22960, 25563}},
 {30332: {29810, 30036, 30304}},
 {1054189: {30078, 30621, 1005871}},
 {25449: {24924}},
 {25182: set()},
 {283312: set()},
 {1121440: set()},
 {25031: {24745}},
 {28184: {27233}},
 {29451: set()},
 {1143741: set()},
 {26742: {26676}},
 {29446: {25327,
   25451,
   25482,
   25532,
   25679,
   26164,
   26186,
   26429,
   26963,
   28052,
   28409,
   28580,
   28591,
   28784,
   28863,
   28864,
   28867,
   28991,
   28997,
   29113,
   29221,
   29267,
   29308,
   138064}},
 {221357: {25043}},
 {30665: {25563, 30155}},
 {28903: {28433}},
 {29452: set()},
 {29140: {26727, 28179, 28369, 28929}},
 {30451: {25563, 30155}},
 {1037693: {259774}},
 {29656: set()},
 {30825: set()},
 {24129: {24112}},
 {160254: {25563, 28104, 28794, 29544}},
 {1050402: {24758}},
 {177728: set()},
 {25139: {25086}},
 {192358: {30077, 30542}},
 {220557: {25182}},
 {30841: set()},
 {28921: {28681}},
 {26854: {24981, 25007, 25057, 25153, 25366, 26221}},
 {26597: {25756}},
 {1

# Guardo el diccionario de las relaciones en realtions.json

In [ ]:

import json

merged = {}
for d in relations:
    merged.update(d)

def convert_sets(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, dict):
        return {k: convert_sets(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_sets(i) for i in obj]
    else:
        return obj

relations_json_ready = convert_sets(merged)

with open("data/relations.json", "w", encoding="utf-8") as f:
    json.dump(relations_json_ready, f, ensure_ascii=False, indent=4)
